#Import libraies and packages

In [ ]:
pip install ray torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import pandas as pd
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest import ConcurrencyLimiter

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Model

In [ ]:
import tensorflow as tf

def ConvNet(config, len_classes=2):
    input = tf.keras.layers.Input(shape=(43893, 1))
    x = input
    x = tf.keras.layers.Conv1D(filters=config['conv_block1_filters'], kernel_size=(8), strides=1)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    if config['fc_layer_type'] == 'dense':
        if config['pool_type'] == 'max':
            x = tf.keras.layers.GlobalMaxPooling1D()(x)
        else:
            x = tf.keras.layers.GlobalAveragePooling1D()(x)

        # Fully connected layer 1
        x = tf.keras.layers.Dense(units=config['fc1_units'])(x)
        x = tf.keras.layers.Dropout(config['dropout_rate'])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)

        # Fully connected layer 2
        x = tf.keras.layers.Dense(units=len_classes)(x)
        x = tf.keras.layers.Dropout(config['dropout_rate'])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        predictions = tf.keras.layers.Dense(1,tf.keras.layers.Activation('sigmoid'))(x)

    else:
        # Fully connected layer 1
        x = tf.keras.layers.Conv1D(filters=config['fc1_units'], kernel_size=1, strides=1)(x)
        x = tf.keras.layers.Dropout(config['dropout_rate'])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)


        # Fully connected layer 2
        x = tf.keras.layers.Conv1D(filters=len_classes, kernel_size=1, strides=1)(x)
        x = tf.keras.layers.Dropout(config['dropout_rate'])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        
        if config['pool_type'] == 'max':
            x = tf.keras.layers.GlobalMaxPooling1D()(x)
        else:
            x = tf.keras.layers.GlobalAveragePooling1D()(x)
        
        predictions = tf.keras.layers.Dense(1,tf.keras.layers.Activation('sigmoid'))(x)
    print(predictions)
    model = tf.keras.Model(inputs=input, outputs=predictions)
    
    print(model.summary())
    print(f'Total number of layers: {len(model.layers)}')

    return model
   

## Metrics

In [ ]:
from keras import backend as K
import random
from sklearn.metrics import f1_score, precision_score, recall_score


def recall_m(y_true, y_pred):
    y_true.numpy()
    y_pred.numpy()
    recall = recall_score(y_true, np.argmax(y_pred, axis = 1), average='weighted', zero_division = 1)
    return recall

def precision_m(y_true, y_pred):
    y_true.numpy()
    y_pred.numpy()
    precision = precision_score(y_true, np.argmax(y_pred, axis = 1), average='weighted', zero_division = 1)
    return precision

def f1_m(y_true, y_pred):
    y_true.numpy()
    y_pred.numpy()
    f1 = f1_score(y_true, np.argmax(y_pred, axis = 1), average='weighted', zero_division = 1)
    return f1

## Training function

In [ ]:
from sklearn import preprocessing

def train_mnist(config):
  path ='/content/drive/MyDrive/ART_Inv/CNN/Ray_Tune/Clinical_data_and_RNA_total_Features_PFS.csv'
  data_frame = pd.read_csv(path)
  
  from sklearn.model_selection import train_test_split
  X = data_frame.iloc[:,28:43921  ]   
  Y=[]
  for i in range (len(data_frame)):
      if data_frame.PFS[i]<3: # If PFS is lower than 3 months, I will consider it as NonResponder (NR)
          Y.append(0)
      # elif data_frame.PFS[i]<6:
      #     Y.append(1)
      else:
          Y.append(1)# If PFS is over 3 months, I will consider it as Responder (R)
  scaler = preprocessing.MinMaxScaler()
  names = X.columns
  d = scaler.fit_transform(X)
  X = pd.DataFrame(d, columns=names)
  XTrain, XTest, yTrain, yTest = train_test_split(X, Y, test_size=0.20, stratify = Y)
  # Convert sets to arrays
  XTrain = XTrain.values
  XTest = XTest.values
  # It is mandatory to transform Y list into array for trainning the model
  yTrain=np.array(yTrain)
  yTest=np.array(yTest)

  X_train = XTrain.reshape(XTrain.shape[0], 43893 , 1)
  X_test = XTest.reshape(XTest.shape[0], 43893, 1)
  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')
  # Create model
  model = ConvNet(config)
  # Compile model with losses and metrics
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate =config['lr']),
                # tf.keras.optimizers.RMSprop(learning_rate =config['lr']),
                  loss='binary_crossentropy',
                  metrics=['accuracy', f1_m, precision_m, recall_m], run_eagerly=True)
  # Start model training
  history_m = model.fit(X_train, yTrain,
                      epochs=100,
                      validation_data=(X_test, yTest))
  history_m = {
  "loss": history_m.history["loss"][0],
  "val_loss": history_m.history["val_loss"][0],
  "accuracy": history_m.history["accuracy"][0],
  "val_accuracy": history_m.history["val_accuracy"][0],
  "val_f1_m": history_m.history["val_f1_m"][0]
  }
  return history_m

# Bayesian Optimization

In [ ]:
pip install hpbandster ConfigSpace

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install ConfigSpace

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import ConfigSpace as CS
config_space = CS.ConfigurationSpace()

In [ ]:
config_space = CS.ConfigurationSpace()
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="lr", choices=[ 0.0001, 0.001, 0.01, 0.1]))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="dropout_rate", choices=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="conv_block1_filters", choices=[8, 16, 32, 64, 128]))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="fc_layer_type", choices= ['dense', 'convolution']))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="pool_type", choices= ['max', 'average']))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="fc1_units", choices=[8, 16, 32, 64, 128]))

# config_space.add_hyperparameter(
#     CS.CategoricalHyperparameter(
#         name="lr", choices=[0.001]))
# config_space.add_hyperparameter(
#     CS.CategoricalHyperparameter(
#         name="dropout_rate", choices=[ 0.3])) 
# config_space.add_hyperparameter(
#     CS.CategoricalHyperparameter(
#         name="conv_block1_filters", choices=[8]))
# config_space.add_hyperparameter(
#     CS.CategoricalHyperparameter(
#         name="fc_layer_type", choices= ['dense']))
# config_space.add_hyperparameter(
#     CS.CategoricalHyperparameter(
#         name="pool_type", choices= [ 'average']))
# config_space.add_hyperparameter(
#     CS.CategoricalHyperparameter(
#         name="fc1_units", choices=[16]))

fc1_units, Type: Categorical, Choices: {8, 16, 32, 64, 128}, Default: 8

In [ ]:
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB
algo = TuneBOHB(
    config_space, metric="val_f1_m", mode="max", max_concurrent=1)
bohb = HyperBandForBOHB(
    time_attr="training_iteration",
    metric="val_f1_m",
      mode="max",
      max_t=2)
for i in range(1):
  tune.run(train_mnist, scheduler=bohb, num_samples=3, search_alg=algo)

2022-11-21 16:06:02,153	WARNING callback.py:109 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


(train_mnist pid=6763) 2022-11-21 16:06:25.376199: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


(train_mnist pid=6763) KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), name='dense_2/activation_2/Sigmoid:0', description="created by layer 'dense_2'")
(train_mnist pid=6763) Model: "model"
(train_mnist pid=6763) _________________________________________________________________
(train_mnist pid=6763)  Layer (type)                Output Shape              Param #   
(train_mnist pid=6763) =================================================================
(train_mnist pid=6763)  input_1 (InputLayer)        [(None, 43893, 1)]        0         
(train_mnist pid=6763)                                                                  
(train_mnist pid=6763)  conv1d (Conv1D)             (None, 43886, 64)         576       
(train_mnist pid=6763)                                                                  
(train_mnist pid=6763)  batch_normalization (BatchN  (None, 43886, 64)        256       
(train_mnist pid=6763)  ormalization)                             

(train_mnist pid=6763) WARNING:tensorflow:AutoGraph could not transform <function f1_m at 0x7f31942b3050> and will run it as-is.
(train_mnist pid=6763) Cause: Unable to locate the source code of <function f1_m at 0x7f31942b3050>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
(train_mnist pid=6763) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(train_mnist pid=6763) WARNING:tensorflow:AutoGraph could not transform <function f1_m at 0x7f31942b3050> and will run it as-is.
(train_mnist pid=6763) Cause: Unable to locate the source code of <function f1_m at 0x7f31942b3050>. Note that functions defined in certain environments, like the interactive Python s

5/5 [==============================] - 13s 2s/step - loss: 0.7063 - accuracy: 0.4653 - f1_m: 0.2885 - precision_m: 0.7656 - recall_m: 0.4500 - val_loss: 0.6954 - val_accuracy: 0.4595 - val_f1_m: 0.1843 - val_precision_m: 0.8755 - val_recall_m: 0.2656
(train_mnist pid=6763) Epoch 2/100
5/5 [==============================] - 14s 3s/step - loss: 0.6940 - accuracy: 0.5625 - f1_m: 0.3074 - precision_m: 0.7531 - recall_m: 0.4750 - val_loss: 0.6957 - val_accuracy: 0.4595 - val_f1_m: 0.1843 - val_precision_m: 0.8755 - val_recall_m: 0.2656
(train_mnist pid=6763) Epoch 3/100
5/5 [==============================] - 13s 3s/step - loss: 0.6627 - accuracy: 0.6319 - f1_m: 0.3077 - precision_m: 0.7535 - recall_m: 0.4750 - val_loss: 0.6977 - val_accuracy: 0.4595 - val_f1_m: 0.1843 - val_precision_m: 0.8755 - val_recall_m: 0.2656
(train_mnist pid=6763) Epoch 4/100
5/5 [==============================] - 12s 2s/step - loss: 0.6659 - accuracy: 0.5903 - f1_m: 0.3233 - precision_m: 0.7566 - recall_m: 0.4875 -

...

5/5 [==============================] - 3s 491ms/step - loss: 0.7492 - accuracy: 0.4861 - f1_m: 0.3261 - precision_m: 0.7713 - recall_m: 0.4812 - val_loss: 0.6936 - val_accuracy: 0.4595 - val_f1_m: 0.4729 - val_precision_m: 0.7994 - val_recall_m: 0.6031
(train_mnist pid=8236) Epoch 2/100
5/5 [==============================] - 2s 484ms/step - loss: 0.7386 - accuracy: 0.4931 - f1_m: 0.3085 - precision_m: 0.7547 - recall_m: 0.4750 - val_loss: 0.6937 - val_accuracy: 0.4595 - val_f1_m: 0.4729 - val_precision_m: 0.7994 - val_recall_m: 0.6031
(train_mnist pid=8236) Epoch 3/100
5/5 [==============================] - 2s 474ms/step - loss: 0.7460 - accuracy: 0.4792 - f1_m: 0.3017 - precision_m: 0.7549 - recall_m: 0.4688 - val_loss: 0.6928 - val_accuracy: 0.4595 - val_f1_m: 0.4729 - val_precision_m: 0.7994 - val_recall_m: 0.6031
(train_mnist pid=8236) Epoch 4/100
5/5 [==============================] - 2s 499ms/step - loss: 0.7232 - accuracy: 0.5556 - f1_m: 0.2979 - precision_m: 0.7602 - recall_m: 

(train_mnist pid=9480) 2022-11-21 16:32:09.709510: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


(train_mnist pid=9480) KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), name='dense/activation_2/Sigmoid:0', description="created by layer 'dense'")
(train_mnist pid=9480) Model: "model"
(train_mnist pid=9480) _________________________________________________________________
(train_mnist pid=9480)  Layer (type)                Output Shape              Param #   
(train_mnist pid=9480) =================================================================
(train_mnist pid=9480)  input_1 (InputLayer)        [(None, 43893, 1)]        0         
(train_mnist pid=9480)                                                                  
(train_mnist pid=9480)  conv1d (Conv1D)             (None, 43886, 8)          72        
(train_mnist pid=9480)                                                                  
(train_mnist pid=9480)  batch_normalization (BatchN  (None, 43886, 8)         32        
(train_mnist pid=9480)  ormalization)                                 

(train_mnist pid=9480) WARNING:tensorflow:AutoGraph could not transform <function f1_m at 0x7f41c0d32050> and will run it as-is.
(train_mnist pid=9480) Cause: Unable to locate the source code of <function f1_m at 0x7f41c0d32050>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
(train_mnist pid=9480) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(train_mnist pid=9480) WARNING:tensorflow:AutoGraph could not transform <function f1_m at 0x7f41c0d32050> and will run it as-is.
(train_mnist pid=9480) Cause: Unable to locate the source code of <function f1_m at 0x7f41c0d32050>. Note that functions defined in certain environments, like the interactive Python s

5/5 [==============================] - 23s 5s/step - loss: 0.6958 - accuracy: 0.4583 - f1_m: 0.3165 - precision_m: 0.7564 - recall_m: 0.4812 - val_loss: 0.6952 - val_accuracy: 0.4595 - val_f1_m: 0.2639 - val_precision_m: 0.7555 - val_recall_m: 0.4344
(train_mnist pid=9480) Epoch 2/100
5/5 [==============================] - 20s 4s/step - loss: 0.6899 - accuracy: 0.5486 - f1_m: 0.3469 - precision_m: 0.7730 - recall_m: 0.5000 - val_loss: 0.6947 - val_accuracy: 0.4595 - val_f1_m: 0.2639 - val_precision_m: 0.7555 - val_recall_m: 0.4344
(train_mnist pid=9480) Epoch 3/100
5/5 [==============================] - 20s 4s/step - loss: 0.6840 - accuracy: 0.5556 - f1_m: 0.2962 - precision_m: 0.7570 - recall_m: 0.4625 - val_loss: 0.6940 - val_accuracy: 0.4595 - val_f1_m: 0.2639 - val_precision_m: 0.7555 - val_recall_m: 0.4344
(train_mnist pid=9480) Epoch 4/100
5/5 [==============================] - 20s 4s/step - loss: 0.6798 - accuracy: 0.6250 - f1_m: 0.3145 - precision_m: 0.7533 - recall_m: 0.4812 -